In [0]:
%pip install -U -qqqq backoff databricks-openai openai uv databricks-agents>=1.0.0 mcp>=1.9 databricks-sdk[openai] databricks-mcp
%restart_python

# MCP Agent Test

In [0]:
# from databricks.sdk import WorkspaceClient
# w = WorkspaceClient()
# w.secrets.put_secret("cindy_demos","cindy_demo_service_principal_secret",string_value ="")
# w.secrets.put_secret("cindy_demos","cindy_demo_service_principal_client",string_value = "")

## Test out custom MCP server hosted through Databricks Apps
- Use Service Principal to initialize Workspace. PAT won't work as Apps does not support authentication through PAT 

### Using SP

In [0]:
from databricks_mcp import DatabricksOAuthClientProvider, DatabricksMCPClient
from databricks.sdk import WorkspaceClient

client_secret_scope_name = "cindy_demos"
client_secret_key_name = "cindy_demo_service_principal_secret"
client_id_key_name = "cindy_demo_service_principal_client"

w = WorkspaceClient(
    host="https://e2-demo-field-eng.cloud.databricks.com/",
    client_id=dbutils.secrets.get(scope=client_secret_scope_name, key=client_id_key_name),
    client_secret=dbutils.secrets.get(scope=client_secret_scope_name, key=client_secret_key_name)
)
mcp_client = DatabricksMCPClient(server_url="https://mcp-custom-server-1444828305810485.aws.databricksapps.com/mcp", workspace_client=w)
tools = await mcp_client._get_tools_async()
tools

In [0]:
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksOAuthClientProvider
from databricks.sdk.credentials_provider import ModelServingUserCredentials
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

async def main():
    app_url = "https://mcp-custom-server-1444828305810485.aws.databricksapps.com/mcp"
    client = WorkspaceClient(
    host="https://e2-demo-field-eng.cloud.databricks.com/",
    client_id=dbutils.secrets.get(scope=client_secret_scope_name, key=client_id_key_name),
    client_secret=dbutils.secrets.get(scope=client_secret_scope_name, key=client_secret_key_name)
)

    async with streamablehttp_client(app_url, auth=DatabricksOAuthClientProvider(client)) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tools = await session.list_tools()
            print(f"Available tools: {[tool.name for tool in tools.tools]}")

await main()

### Using PAT

In [0]:
import asyncio
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient

async def get_tools_async(mcp_client):
    return await mcp_client._get_tools_async()

async def main():
    app_url = "https://mcp-calender-1444828305810485.aws.databricksapps.com/mcp/"
    client = WorkspaceClient(
    )

    async with streamablehttp_client(app_url, auth=DatabricksOAuthClientProvider(client)) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tools = await session.list_tools()
            print(f"Available tools: {[tool.name for tool in tools.tools]}")

# Ensure the following line is not indented
await main()

## Test out agent endpoint with OBO

### Using PAT
- Agent uses a Oauth token with minimal scope of the notebook user, therefore can access custom MCP server (if notebook user has access)

In [0]:
from databricks.sdk import WorkspaceClient
endpoint = "agents_cindy_demo_catalog-movies-langgraph-mcp-movie-agent-obo"

w = WorkspaceClient()

client = w.serving_endpoints.get_open_ai_client()
resp = client.responses.create(
    model=endpoint,
    input=[{"role": "user", "content": "what tools do i have"}]
)
print(resp.output[0].content[0].text)

### Using SP

In [0]:
from databricks.sdk import WorkspaceClient
endpoint = "agents_cindy_demo_catalog-movies-langgraph-mcp-movie-agent-obo"
client_secret_scope_name = "cindy_demos"
client_secret_key_name = "cindy_demo_service_principal_secret"
client_id_key_name = "cindy_demo_service_principal_client"
endpoint = "agents_cindy_demo_catalog-movies-langgraph-mcp-movie-agent-obo"

w = WorkspaceClient(
    host="https://e2-demo-field-eng.cloud.databricks.com/",
    client_id=dbutils.secrets.get(scope=client_secret_scope_name, key=client_id_key_name),
    client_secret=dbutils.secrets.get(scope=client_secret_scope_name, key=client_secret_key_name)
)

client = w.serving_endpoints.get_open_ai_client()

resp = client.responses.create(
    model=endpoint,
    input=[{"role": "user", "content": "what tools do i have"}],
)
print(resp.output[0].content[0].text)

In [0]:
from databricks.sdk import WorkspaceClient
client_secret_scope_name = "cindy_demos"
client_secret_key_name = "cindy_demo_service_principal_secret"
client_id_key_name = "cindy_demo_service_principal_client"
endpoint = "agents_cindy_demo_catalog-movies-langgraph-mcp-movie-agent-obo"

w = WorkspaceClient(
    host="https://e2-demo-field-eng.cloud.databricks.com/",
    client_id=dbutils.secrets.get(scope=client_secret_scope_name, key=client_id_key_name),
    client_secret=dbutils.secrets.get(scope=client_secret_scope_name, key=client_secret_key_name)
)

# OpenAI-compatible client bound to your workspace
client = w.serving_endpoints.get_open_ai_client()

# Agent Framework expects `input` (array of chat messages), and `max_output_tokens`
resp = client.responses.create(
    model=endpoint,
    input=[{"role": "user", "content": "what is budget for interstellar"}],
    temperature=0.2,
    max_output_tokens=512,
)

# Print the text output
# print(resp.output[0].content)